In [1]:
from chrome_crawler import Chrome_Crawler
from lib import *

In [2]:
PERSON_XPATH = {
    'name':'/html[1]/body[1]/div[1]/div[3]/div[2]/table[2]/tbody[1]/tr[1]/td[1]/h1[1]',
    'role':'/html[1]/body[1]/div[1]/div[3]/div[2]/h2[1]',
    'movie': {
        'date'          :'/html[1]/body[1]/div[1]/div[3]/div[2]/table[2]/tbody[1]/tr[1]/td[1]/table[1]/tbody[1]/tr/td[1]/font[1]',
        'title'         :'/html[1]/body[1]/div[1]/div[3]/div[2]/table[2]/tbody[1]/tr[1]/td[1]/table[1]/tbody[1]/tr/td[2]/font[1]/a[1]',
        'studio'        :'/html[1]/body[1]/div[1]/div[3]/div[2]/table[2]/tbody[1]/tr[1]/td[1]/table[1]/tbody[1]/tr/td[3]/font[1]/a[1]',
        'lifetimeGross' :'/html[1]/body[1]/div[1]/div[3]/div[2]/table[2]/tbody[1]/tr[1]/td[1]/table[1]/tbody[1]/tr/td[4]/font[1]',
        'opening'       :'/html[1]/body[1]/div[1]/div[3]/div[2]/table[2]/tbody[1]/tr[1]/td[1]/table[1]/tbody[1]/tr/td[6]/font[1]',
    },
    'average':'/html[1]/body[1]/div[1]/div[3]/div[2]/table[2]/tbody[1]/tr[1]/td[1]/font[2]/b[1]'
}
PEOPLE = 'https://www.boxofficemojo.com/people/chart/?view=&id=%s.htm'

In [3]:
_chrome_option = webdriver.ChromeOptions()
_chrome_option.add_argument("headless")
chrome_option = _chrome_option
driver = webdriver.Chrome(chrome_options=chrome_option)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [4]:
def get_list_person():
    # Get json movie list from folder Movies
    listMovies = os.listdir('Movies')
    listPeople = []

    for movie in listMovies:
        # Verify movie is .json file
        if (not(movie.endswith('.json'))):
            break

        # Create link of movie
        url = 'Movies\\%s' % movie
        # Get movie information
        movieInfo = json.load(open(url, 'r'))
        # Get list persons from movieInfo
        listPeople += movieInfo['director'] + movieInfo['writer'] + movieInfo['actor'] + movieInfo['producer']

    # Remove duplicated values
    people = np.unique(listPeople)
    return people

In [5]:
def get_person_info(url):
    driver.get(url)
    # driver.get(linkGenre)

    id = url.split('id=')[1][:-4]
    
    try:
        name = driver.find_element_by_xpath(PERSON_XPATH['name']).text
        average = driver.find_element_by_xpath(PERSON_XPATH['average']).text.split('$')[1]
        role = driver.find_element_by_xpath(PERSON_XPATH['role']).text.split('> ')[2]
   

        elements = driver.find_elements_by_xpath(PERSON_XPATH['movie']['date'])[1:]
        date = [e.text for e in elements]


        elements = driver.find_elements_by_xpath(PERSON_XPATH['movie']['title'])[1:]
        title = [e.get_attribute('href')[41:-4] for e in elements]


        elements = driver.find_elements_by_xpath(PERSON_XPATH['movie']['studio'])[1:]
        studio = [e.get_attribute('href')[51:-4] for e in elements]

        elements = driver.find_elements_by_xpath(PERSON_XPATH['movie']['lifetimeGross'])[1:]
        lifetimeGross = [e.text[1:].replace(',','') for e in elements]

        elements = driver.find_elements_by_xpath(PERSON_XPATH['movie']['opening'])[1:]
        opening = [e.text[1:].replace(',','') for e in elements]
    except:
        return {
            'id':id,
            'name':'',
            'role':'',
            'average':'',
            'movies':''
        }


    movies = pd.DataFrame({
        'date': date,
        'title': title,
        'studio': studio,
        'lifetimeGross': lifetimeGross,
        'opening': opening
    }).to_dict('records')

    person = {
        'id':id,
        'name':name,
        'role':role,
        'average':average,
        'movies':movies
    }

    return person


In [6]:
chrome_crawler = Chrome_Crawler()

persons = get_list_person()
len(np.unique(persons))

735

In [7]:
file = open('more_people.txt', 'r')
arr = file.read()
# arr = file.readlines()
# arr.remove('\n')
arr.split('\n')
file.close()

In [8]:
def get_more_person_urls():
    file = open('more_people.txt', 'r')
    data = file.read()
    file.close()
    
    persons = data.split('\n')
    listUrls = []
    for person in persons:
#         listUrls += (PEOPLE % person)
        listUrls.append(PEOPLE % person)
    
    return listUrls
    

In [9]:
more_person_url = get_more_person_urls()
len(more_person_url)
morePersons = np.unique(more_person_url)
len(morePersons)
morePersons
morePersons = morePersons[1:]

In [10]:
people = np.concatenate((morePersons, persons), axis = 0)
len(people)

1630

In [11]:
_round = 0

while _round < len(people):
    person = get_person_info(people[_round])
    id = person['id']
    if (person['name'] == ''):
        print('exception: %s' % id)
        with open('PersonExceptions.txt', 'a+') as f_except:
            f_except.write(id + '\n')
        _round += 1
        continue
            
    try:
        with open('Persons/%s.json' % id, 'w') as fp:
            json.dump(person, fp)

        time.sleep(np.random.randint(1,4)/4)

        # Reset driver:
        if ((_round % 50 == 0) or (_round == len(people))):
            print ("Crawling person...\t%d/%d\n"%(_round, len(people)))
            chrome_crawler.restart_crawler(delay_time=15)
    except:
        print('exception: %s' % id)
        with open('PersonExceptions.txt', 'a+') as f_except:
            f_except.write(id + '\n')
            
    _round += 1

Crawling person...	0/1630

Crawling person...	50/1630

Crawling person...	100/1630

Crawling person...	150/1630

exception: colinwilson
Crawling person...	200/1630

Crawling person...	250/1630

Crawling person...	300/1630

Crawling person...	350/1630

Crawling person...	400/1630

Crawling person...	450/1630

Crawling person...	500/1630

Crawling person...	550/1630

Crawling person...	600/1630

Crawling person...	650/1630

Crawling person...	700/1630

Crawling person...	750/1630

Crawling person...	800/1630

Crawling person...	850/1630

Crawling person...	900/1630

Crawling person...	950/1630

Crawling person...	1000/1630

Crawling person...	1050/1630

Crawling person...	1100/1630

Crawling person...	1150/1630

Crawling person...	1200/1630

Crawling person...	1250/1630

Crawling person...	1300/1630

Crawling person...	1350/1630

Crawling person...	1400/1630

Crawling person...	1450/1630

Crawling person...	1500/1630

Crawling person...	1550/1630

Crawling person...	1600/1630



In [13]:
len(os.listdir('Persons'))

1539